In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats
from math import exp, log, sqrt

In [17]:
ar=0.15

R=log(1+ar)/260
v=ar/sqrt(260)

assets = np.empty((250,25))
daily_returns = np.empty((250,25))
for i in range(250):
    price=np.empty(25)
    price[0]=100
    dr=np.empty(25)
    dr[0]=0
    for t in range(1,25):
        price[t]=price[t-1]*exp(R+v*np.random.normal(0,1))
        dr=log(price[t]/price[t-1])
    assets[i]=price
    daily_returns[i]=dr

In [21]:
final_prices = assets[:,-1]
rates_of_return = ((final_prices/assets[:,0])-1)*100
mean_annualized_RoR = np.mean(rates_of_return)*(260/24)
stdev_annualized_RoR = np.std(rates_of_return)*sqrt(260/24)

In [25]:
vol=np.empty(250)
for i in range(250):
    vol[i]=np.std(daily_returns[i])

mean_annualized_volatility=np.mean(vol)*sqrt(260/24)
stdev_annualized_volatility=np.std(vol)*sqrt(260/24)

In [26]:
print('\t\t   Mean\t\tStd. Dev.\n')
print(f'Final Price:       {np.mean(final_prices):.2f}\t{np.std(final_prices):.2f}')
print(f'Rates of Return:   {mean_annualized_RoR:.2f}\t{stdev_annualized_RoR:.2f}')
print(f'Volatility:        {mean_annualized_volatility:.3f}\t{stdev_annualized_volatility:.3f}')

		   Mean		Std. Dev.

Final Price:       101.36	4.67
Rates of Return:   14.72	15.37
Volatility:        0.000	0.000


In [24]:
def bscall(price,strike,rfr,vol,div,T):
    N=stats.norm.cdf
    d1=(np.log(price/strike)+(rfr-div+vol**2/2)*T)/(vol*np.sqrt(T))
    d2=d1-vol*np.sqrt(T)
    call = price*np.exp(-div*T)*N(d1)-strike*np.exp(-rfr*T)*N(d2)
    return call

In [29]:
rfr=0.05
div=0
vol=0.15

strikes=[97,100,103,105]
option_prices=np.empty((4,250,25))
for k in strikes:
    for i in range(250):
        calls=np.empty(25)
        for j in range(25):
            T=(25-j)
            price=assets[i,j]
            calls[j]=bscall(price,k,rfr,vol,div,T/260)
        option_prices[strikes.index(k)][i]=calls

In [44]:
avg_option_prices=np.empty(4)
stdev_option_prices=np.empty(4)
for i in range(4):
    avg_option_prices[i]=np.mean(option_prices[i])
    stdev_option_prices[i]=np.std(option_prices[i])
print('\tInitial Price\t\tFinal Value\n')
print('\n\t\t\t   Mean\t\tStd. Dev.\n')
for i in range(4):
    print(f'K={strikes[i]}:\t   {np.mean(option_prices[i,:,0]):.2f}     \t {avg_option_prices[i]:.2f}\t {stdev_option_prices[i]:.2f}')


	Initial Price		Final Value


			   Mean		Std. Dev.

K=97:	   4.06     	 4.43	 2.84
K=100:	   2.10     	 2.37	 2.23
K=103:	   0.89     	 1.06	 1.51
K=105:	   0.44     	 0.56	 1.06


In [45]:
inTheMoney = np.empty((4,250,2))
for i in range(4):
    for p in range(250):
        if final_prices[p]>strikes[i]:
            inTheMoney[i][p][0]=1
            inTheMoney[i][p][1]=final_prices[p]-strikes[i]
        else:
            inTheMoney[i][p][0]=0
            inTheMoney[i][p][1]=0


avg_ITM=np.empty(4)
stdev_ITM=np.empty(4)
num_ITM=np.empty(4)
for i in range(4):
    avg_ITM[i]=np.mean(inTheMoney[i,:,1]*inTheMoney[i,:,0])
    stdev_ITM[i]=np.std(inTheMoney[i,:,0]*inTheMoney[i,:,1])
    num_ITM[i]=np.sum(inTheMoney[i,:,0])

print('\n\t\t   Number\t\tMean\t\tStd. Dev.\n')
for i in range(4):
    print(f'K={strikes[i]}:\t   {num_ITM[i]:.2f}\t\t{avg_ITM[i]:.2f}\t\t{stdev_ITM[i]:.2f}')





		   Number		Mean		Std. Dev.

K=97:	   209.00		4.77		4.05
K=100:	   156.00		2.58		3.28
K=103:	   81.00		1.19		2.29
K=105:	   55.00		0.66		1.61
